<a href="https://colab.research.google.com/github/Dash400air/Bert_task/blob/main/STS_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [3]:
!pip install sentence-transformers

     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 1.2 MB 17.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=43ec3b9378c33f9ef7a32a5769ac4198308f5052de0683d37a95f8ddf68f8373
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers


In [5]:
import pandas as pd
import numpy as np

import torch

from sentence_transformers import SentenceTransformer

In [6]:
!wget "http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz"

--2021-09-16 06:29:04--  http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.es (ixa2.si.ehu.es)... 158.227.106.100
Connecting to ixa2.si.ehu.es (ixa2.si.ehu.es)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz [following]
--2021-09-16 06:29:05--  http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.eus (ixa2.si.ehu.eus)... 158.227.106.100
Connecting to ixa2.si.ehu.eus (ixa2.si.ehu.eus)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409630 (400K) [application/x-gzip]
Saving to: ‘Stsbenchmark.tar.gz’

Stsbenchmark.tar.gz 100%[===================>] 400.03K   422KB/s    in 0.9s    

2021-09-16 06:29:07 (422 KB/s) - ‘Stsbenchmark.tar.gz’ saved [409630/409630]



In [7]:
!tar -xzvf "/content/Stsbenchmark.tar.gz"

stsbenchmark/
stsbenchmark/readme.txt
stsbenchmark/sts-test.csv
stsbenchmark/correlation.pl
stsbenchmark/LICENSE.txt
stsbenchmark/sts-dev.csv
stsbenchmark/sts-train.csv


In [46]:
train = pd.read_csv("/content/stsbenchmark/sts-train.csv", sep='\t', 
                    header=None, error_bad_lines=False, 
                    names=['type', 'source', 'ver', 'num', 'score', 's1', 's2'])
valid = pd.read_csv("/content/stsbenchmark/sts-dev.csv", sep='\t', 
                    header=None, error_bad_lines=False, 
                    names=['type', 'source', 'ver', 'num', 'score', 's1', 's2'])

# Preprocessing

In [47]:
train.head()

,type,source,ver,num,score,s1,s2
0,main-captions,MSRvid,2012test,1,5.00,A plane is taking off.,An air plane is taking off.
1,main-captions,MSRvid,2012test,4,3.80,A man is playing a large flute.,A man is playing a flute.
2,main-captions,MSRvid,2012test,5,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,main-captions,MSRvid,2012test,6,2.60,Three men are playing chess.,Two men are playing chess.
4,main-captions,MSRvid,2012test,9,4.25,A man is playing the cello.,A man seated is playing the cello.


In [49]:
valid.head()

,type,source,ver,num,score,s1,s2
0,main-captions,MSRvid,2012test,0,5.00,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.
1,main-captions,MSRvid,2012test,2,4.75,A young child is riding a horse.,A child is riding a horse.
2,main-captions,MSRvid,2012test,3,5.00,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.
3,main-captions,MSRvid,2012test,7,2.40,A woman is playing the guitar.,A man is playing guitar.
4,main-captions,MSRvid,2012test,8,2.75,A woman is playing the flute.,A man is playing a flute.


# Vectorizing

In [19]:
model = SentenceTransformer('all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [50]:
def get_vectors(df):
    s1 = df['s1'].values
    s2 = df['s2'].values

    s1_vectors = model.encode(s1)
    s2_vectors = model.encode(s2)

    return s1_vectors, s2_vectors

In [52]:
train_s1_vectors, train_s2_vectors = get_vectors(train)
valid_s1_vectors, valid_s2_vectors = get_vectors(valid)

# Get Cosine Similarity

In [53]:
def get_cossimlarity(s1_vectors, s2_vectors):
    s1_tensor = torch.from_numpy(s1_vectors).clone()
    s2_tensor = torch.from_numpy(s2_vectors).clone()

    sims = []
    for s1_vec, s2_vec in zip(s1_tensor, s2_tensor):
        sim = torch.cosine_similarity(s1_vec, s2_vec, dim=0)
        sim = sim.detach().numpy().copy()
        sims.append(sim)
    return sims

In [54]:
train_sims = get_cossimlarity(train_s1_vectors, train_s2_vectors)
valid_sims = get_cossimlarity(valid_s1_vectors, valid_s2_vectors)

In [55]:
train['cos_similarity'] = train_sims
valid['cos_similarity'] = valid_sims

In [56]:
train.head()

,type,source,ver,num,score,s1,s2,cos_similarity
0,main-captions,MSRvid,2012test,1,5.00,A plane is taking off.,An air plane is taking off.,0.95965767
1,main-captions,MSRvid,2012test,4,3.80,A man is playing a large flute.,A man is playing a flute.,0.8690923
2,main-captions,MSRvid,2012test,5,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...,0.87193775
3,main-captions,MSRvid,2012test,6,2.60,Three men are playing chess.,Two men are playing chess.,0.8031405
4,main-captions,MSRvid,2012test,9,4.25,A man is playing the cello.,A man seated is playing the cello.,0.90234494


In [57]:
valid.head()

,type,source,ver,num,score,s1,s2,cos_similarity
0,main-captions,MSRvid,2012test,0,5.00,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.,0.9967053
1,main-captions,MSRvid,2012test,2,4.75,A young child is riding a horse.,A child is riding a horse.,0.9509652
2,main-captions,MSRvid,2012test,3,5.00,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.,0.85433936
3,main-captions,MSRvid,2012test,7,2.40,A woman is playing the guitar.,A man is playing guitar.,0.59489024
4,main-captions,MSRvid,2012test,8,2.75,A woman is playing the flute.,A man is playing a flute.,0.73564374


# Linear Regression

In [99]:
from sklearn.linear_model import LinearRegression

In [63]:
X_train, X_valid = train['cos_similarity'].values.reshape(-1, 1), valid['cos_similarity'].values.reshape(-1, 1)
y_train, y_valid = train['score'].values.reshape(-1, 1), valid['score'].values.reshape(-1, 1)

In [59]:
lr = LinearRegression()

In [64]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [66]:
pred = lr.predict(X_valid)

# Score

In [100]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, spearmanr

In [71]:
RMSE = np.sqrt(mean_squared_error(y_valid, pred))
print(RMSE)

0.7361655425935434


In [96]:
y_pred = pred.squeeze()

In [101]:
print(spearmanr(y_valid.squeeze(), y_pred))
print(pearson_corr(y_valid.squeeze(), y_pred))

SpearmanrResult(correlation=0.8743784663328783, pvalue=0.0)
0.872128594102961
